## Import Packages

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Tugas/Capstone/dataset2.csv'
data = pd.read_csv(file_path)
df = data.copy()
df = pd.DataFrame(df)
df.head()

Mounted at /content/drive


,soal_id,isi_soal,tingkat_kesulitan,nama_materi,user_id,opsi_a,opsi_b,jawaban_user,jawaban_benar
0,101,Apa tujuan utama dari sebuah fungsi dalam Python?,mudah,fungsi,1.0,Untuk mengelompokkan kode yang sering digunakan,Untuk mendefiniskan kelas,Untuk mengelompokkan kode yang sering digunakan,Untuk mengelompokkan kode yang sering digunakan
1,102,Bagaimana cara mendefinisikan sebuah fungsi da...,mudah,fungsi,1.0,create function nama_fungsi(),def nama_fungsi(),create function nama_fungsi(),def nama_fungsi()
2,103,Apa yang terjadi jika sebuah fungsi tidak meng...,mudah,fungsi,2.0,Terjadi error,Fungsi akan mengembalikan nilai None,Terjadi error,Fungsi akan mengembalikan nilai None
3,104,Apa yang dimaksud dengan argumen dalam sebuah ...,mudah,fungsi,2.0,Nilai yang diberikan kepada fungsi saat dipanggil,Nilai yang dikembalikan oleh fungsi,Nilai yang diberikan kepada fungsi saat dipanggil,Nilai yang diberikan kepada fungsi saat dipanggil
4,105,Apa perbedaan antara argumen posisi dan argume...,mudah,fungsi,3.0,"Argumen posisi ditentukan oleh urutannya, seda...","Argumen posisi selalu bertipe integer, sedangk...","Argumen posisi ditentukan oleh urutannya, seda...","Argumen posisi ditentukan oleh urutannya, seda..."


### Data Cleaning

In [ ]:
num_duplicates = df.duplicated().sum()
print("jumlah duplikat:{}".format(num_duplicates))

jumlah duplikat:0


In [ ]:
mising_value = df.isnull().sum()
print("jumlah missing value:{}".format(mising_value))

jumlah missing value:soal_id              0
isi_soal             0
tingkat_kesulitan    0
nama_materi          0
user_id              9
opsi_a               0
opsi_b               1
jawaban_user         0
jawaban_benar        0
dtype: int64


In [ ]:
df['user_id'] = df['user_id'].fillna(0)
df['opsi_b'] = df['opsi_b'].fillna('')
df['jawaban_user'] = df['jawaban_user'].fillna('')
df['tingkat_kesulitan'] = df['tingkat_kesulitan'].fillna('')

In [ ]:
mising_value = df.isnull().sum()
print("jumlah missing value:{}".format(mising_value))

jumlah missing value:soal_id              0
isi_soal             0
tingkat_kesulitan    0
nama_materi          0
user_id              0
opsi_a               0
opsi_b               0
jawaban_user         0
jawaban_benar        0
dtype: int64


In [ ]:
df['user_id'] = df['user_id'].astype(int)

## Data Preprocessing

In [ ]:
label_encoder = LabelEncoder()
df['id_materi'] = label_encoder.fit_transform(df['nama_materi']) + 1
df['tingkat_kesulitan'] = label_encoder.fit_transform(df['tingkat_kesulitan']) + 1
df['hasil'] = np.where(df['jawaban_user'] == df['jawaban_benar'], 'benar', 'salah')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   soal_id            149 non-null    int64 
 1   isi_soal           149 non-null    object
 2   tingkat_kesulitan  149 non-null    int64 
 3   nama_materi        149 non-null    object
 4   user_id            149 non-null    int64 
 5   opsi_a             149 non-null    object
 6   opsi_b             149 non-null    object
 7   jawaban_user       149 non-null    object
 8   jawaban_benar      149 non-null    object
 9   id_materi          149 non-null    int64 
 10  hasil              149 non-null    object
dtypes: int64(4), object(7)
memory usage: 12.9+ KB


In [ ]:
data = df.drop(['soal_id','nama_materi','opsi_a','opsi_b','jawaban_user','jawaban_benar'], axis=1)
data.head()

,isi_soal,tingkat_kesulitan,user_id,id_materi,hasil
0,Apa tujuan utama dari sebuah fungsi dalam Python?,1,1,2,benar
1,Bagaimana cara mendefinisikan sebuah fungsi da...,1,1,2,salah
2,Apa yang terjadi jika sebuah fungsi tidak meng...,1,2,2,salah
3,Apa yang dimaksud dengan argumen dalam sebuah ...,1,2,2,benar
4,Apa perbedaan antara argumen posisi dan argume...,1,3,2,benar


In [ ]:
# Buat kolom baru 'jumlah_salah' dengan nilai 1 jika hasil 'salah', 0 jika 'benar'
data['jumlah_salah'] = data['hasil'].apply(lambda x: 1 if x == 'salah' else 0)
# Hitung rata-rata kesalahan dan tingkat kesulitan untuk setiap user per materi
fitur_data = data.groupby(['user_id', 'id_materi']).agg({
    'jumlah_salah': 'mean',  # Rata-rata kesalahan per materi
    'tingkat_kesulitan': 'mean'  # Rata-rata tingkat kesulitan soal yang dikerjakan user
}).reset_index()

In [ ]:
# Label: jika rata-rata kesalahan > 0.5, maka materi dianggap perlu diulang (1)
fitur_data['review_kembali'] = fitur_data['jumlah_salah'].apply(lambda x: 1 if x > 0.5 else 0)

# Tampilkan data yang sudah diolah
print(fitur_data)
print(fitur_data['review_kembali'].value_counts())

    user_id  id_materi  jumlah_salah  tingkat_kesulitan  review_kembali
0         0          4      0.000000           3.000000               0
1         0          5      0.250000           2.750000               0
2         0          6      1.000000           3.000000               1
3         0          7      0.333333           1.666667               0
4         1          1      0.400000           1.800000               0
5         1          2      0.400000           1.800000               0
6         1          3      0.800000           1.800000               1
7         1          4      0.200000           1.800000               0
8         1          5      0.600000           1.800000               1
9         1          6      0.800000           1.800000               1
10        1          7      0.200000           1.800000               0
11        2          1      0.400000           1.800000               0
12        2          2      0.400000           1.800000         

In [ ]:
## Melatih Model

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Fitur dan label
X = fitur_data[['jumlah_salah', 'tingkat_kesulitan']]  # Fitur yang digunakan
y = fitur_data['review_kembali']  # Label

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Buat dan latih model Logistic Regression
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

## Evaluasi Model

In [ ]:
y_pred = model.predict(X_test)

# Evaluasi menggunakan metrik yang lebih mendalam
akurasi = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Tampilkan hasil evaluasi
print(f'Akurasi Model: {akurasi * 100:.2f}%\n')
print('Confusion Matrix:')
print(conf_matrix)
print('\nLaporan Klasifikasi:')
print(class_report)

Akurasi Model: 100.00%

Confusion Matrix:
[[6 0]
 [0 4]]

Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         4

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [ ]:
# Fungsi untuk memberikan rekomendasi berdasarkan data user baru
def rekomendasi_materi(user_id, model):
    """
    Fungsi untuk memberikan rekomendasi apakah user perlu mengulang materi.
    user_data: Data performa user (DataFrame)
    model: Model machine learning yang sudah dilatih
    """
    # Ambil fitur yang diperlukan
    fitur_baru = user_id[['jumlah_salah', 'tingkat_kesulitan']]

    # Prediksi menggunakan model
    prediksi = model.predict(fitur_baru)

    # Tambahkan kolom prediksi ke dalam data
    user_id['review_kembali'] = prediksi

    # Tampilkan rekomendasi
    rekomendasi = user_id[user_id['review_kembali'] == 1]
    if not rekomendasi.empty:
        print("User perlu mengulang materi berikut:")
        for idx, row in rekomendasi.iterrows():
            print(f"- Materi ID: {row['materi_id']} (Rata-rata Kesalahan: {row['jumlah_salah']:.2f})")
    else:
        print(f"Selamat, Anda telah lulus dari latihan akhir ini.")

    return user_id


In [ ]:
# Simulasi data performa user baru
data_user_baru = {
    'user_id': [5, 5, 5, 5 , 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,5],
    'materi_id': [101, 102, 106,110,111,123,124,128,131,133,140,144,150,160,168],
    'jumlah_salah': [0.7, 0.9, 0.7,0.6,0.3,0.5,0.8,0.7,0.8,0.7,0.4,0.5,0.4,0.8,0.7],  # Rata-rata kesalahan per materi
    'tingkat_kesulitan': [3, 3, 3,2,2,2,1,1,1,3,3,1,2,2,1,]  # Rata-rata tingkat kesulitan soal yang dikerjakan
}

# Buat DataFrame dari data user baru
df_user_baru = pd.DataFrame(data_user_baru)

# Gunakan model untuk memberikan rekomendasi pada user baru
hasil_rekomendasi = rekomendasi_materi(df_user_baru, model)
print("\nHasil prediksi dan rekomendasi untuk user baru:")
print(hasil_rekomendasi)


User perlu mengulang materi berikut:
- Materi ID: 102.0 (Rata-rata Kesalahan: 0.90)
- Materi ID: 124.0 (Rata-rata Kesalahan: 0.80)
- Materi ID: 128.0 (Rata-rata Kesalahan: 0.70)
- Materi ID: 131.0 (Rata-rata Kesalahan: 0.80)
- Materi ID: 160.0 (Rata-rata Kesalahan: 0.80)
- Materi ID: 168.0 (Rata-rata Kesalahan: 0.70)

Hasil prediksi dan rekomendasi untuk user baru:
    user_id  materi_id  jumlah_salah  tingkat_kesulitan  review_kembali
0         5        101           0.7                  3               0
1         5        102           0.9                  3               1
2         5        106           0.7                  3               0
3         5        110           0.6                  2               0
4         5        111           0.3                  2               0
5         5        123           0.5                  2               0
6         5        124           0.8                  1               1
7         5        128           0.7                  1 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
